# 🚀 **Passo 5: Comparação Final e Decisão**

## **Aula 5.1: O Momento da Verdade**

---

### **Tá, mas o que é comparação final?**

Imagina que você tá comprando um carro e testou três opções diferentes. Agora chegou a hora de reunir todas as informações: qual é mais rápido, qual é mais econômico, qual é mais confiável, e qual oferece o melhor custo-benefício. É exatamente isso que vamos fazer aqui - só que em vez de carros, são modelos de IA! 😄

**Por que comparação final é importante?**

Até agora medimos velocidade, qualidade e custo separadamente. Mas na vida real, você precisa tomar uma decisão baseada em TUDO junto. É como escolher um restaurante: não é só o preço, não é só a velocidade, não é só a qualidade - é a combinação de tudo!

### **O Que Você Vai Ganhar Desta Análise**

Este notebook produz dois artefatos chave:

1. **Relatório PDF de Análise**: Um documento abrangente com visualizações comparando modelos em todas as dimensões
2. **Resumo CSV**: Dados brutos pra análise adicional ou integração com outras métricas de negócio

Vamos começar consolidando nossos resultados de avaliação!

---

**🖼️ Sugestão de imagem**: Um gráfico de radar mostrando múltiplas dimensões de performance

In [ ]:
# 🛠️ IMPORTANDO AS FERRAMENTAS NECESSÁRIAS
import json
import boto3
import numpy as np
from scipy import stats
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from matplotlib.backends.backend_pdf import PdfPages
import datetime
import sys
from IPython.display import display

sys.path.append("../")

from src import pricing
from src import generate_analysis_report

print("✅ Ferramentas importadas! Vamos fazer a comparação final!")

In [ ]:
# ⚙️ CONFIGURAÇÃO AWS
account_id = boto3.client('sts').get_caller_identity().get('Account')

BUCKET_NAME = f"genai-evaluation-migration-bucket-{account_id}"
PREFIX = "genai_migration"

print(f"🏢 Account ID: {account_id}")
print(f"🪣 Bucket: {BUCKET_NAME}")
print(f"📁 Prefix: {PREFIX}")

### **Carregando Dados de Avaliação**

#### **Recuperando Nossas Informações de Tracking**

Antes de analisar nossos modelos, precisamos acessar as informações de tracking mantidas durante todo o processo de avaliação. Carregando essas informações de tracking primeiro, estabelecemos uma base pra nossa análise consolidada e garantimos que estamos trabalhando com metadados consistentes dos modelos durante todo o processo de comparação.

> **💡 Nota para Aprendizes Autodidatas**: Se você modificou qualquer caminho ou nome de arquivo durante os passos anteriores, certifique-se de que essas mudanças estão refletidas no caminho do arquivo de tracking abaixo. O processo de avaliação depende de informações de tracking consistentes em todos os notebooks.

In [ ]:
# 📊 CARREGANDO NOSSO TRACKING
evaluation_tracking_file = '../data/evaluation_tracking.csv'
evaluation_tracking = pd.read_csv(evaluation_tracking_file)
display(evaluation_tracking)

print("\n💡 Perfeito! Agora temos nosso plano de avaliação carregado.")

### **Preparação da Análise de Custos**

#### **Calculando Custos por Requisição e Projeções**

Enquanto estamos aguardando nossos jobs de avaliação de qualidade LLM-as-a-Judge completarem, podemos analisar o impacto econômico de cada opção de modelo. Considerações de custo são cruciais pra implantações de produção, já que mesmo pequenas diferenças por requisição podem resultar em despesas operacionais significativas em escala.

Nossa análise de custo inclui dois componentes chave:

1. **Custo por Inferência**: Calculado baseado no uso de tokens da nossa avaliação de latência
   custo_por_inferencia = (tokens_entrada × preço_token_entrada) + (tokens_saída × preço_token_saída)
   
   Essas informações de custo serão cruciais ao fazer a seleção final do modelo, permitindo-nos balancear performance e qualidade contra restrições orçamentárias. Em ambientes de produção, essa análise pode ser estendida para incluir custos mensais projetados baseados em volumes de requisição esperados.

2. **Custos de Serviços Auxiliares**: Despesas adicionais além da inferência direta do modelo
   - **LLM-as-a-Judge**: Cobrado baseado no uso do modelo avaliador
   - **Otimização de Prompt**: Cobrado por token para prompts de entrada e otimizados

> **💡 Nota do Workshop**: Os preços do AWS Bedrock são atualizados periodicamente. Para os preços mais atuais, consulte a [Página de Preços do Bedrock](https://aws.amazon.com/bedrock/pricing/).

In [ ]:
# �� CONFIGURANDO O DIRETÓRIO
directory = "../outputs"
print(f"�� Diretório de saída: {directory}")

In [ ]:
# 💰 CALCULANDO CUSTOS PRA TODOS OS MODELOS
calculator = pricing.PriceCalculator()

# Encontrando todos os arquivos CSV correspondentes
all_files = glob.glob(os.path.join(directory, "document_summarization_*.csv"))

print("💰 CALCULANDO CUSTOS...")
print("=" * 40)

# Processando cada arquivo
for filename in all_files:
    print(f"\n📊 Processando {os.path.basename(filename)}...")
    
    # Lendo o arquivo CSV
    document_summarization_df = pd.read_csv(filename)

    model_id = document_summarization_df["model"][0]  # Mudando o nome de volta pra combinar com a config de preços
    
    print(f"🎯 Modelo: {model_id}")
    print(f"�� Total de linhas: {len(document_summarization_df)}")
    
    # Calculando custos de entrada para todas as linhas de uma vez
    input_costs = document_summarization_df["model_input_tokens"].apply(
        lambda tokens: calculator.calculate_input_price(tokens, model_id)
    )

    # Calculando custos de saída para todas as linhas de uma vez
    output_costs = document_summarization_df["model_output_tokens"].apply(
        lambda tokens: calculator.calculate_output_price(tokens, model_id)
    )
    
    # Calculando custos totais
    document_summarization_df["cost"] = (input_costs + output_costs).round(6)

    # Escrevendo de volta no mesmo arquivo
    document_summarization_df.to_csv(filename, index=False)
    print(f"✅ Custos calculados e salvos!")
    
    # Mostrando estatísticas rápidas
    avg_cost = document_summarization_df["cost"].mean()
    total_cost = document_summarization_df["cost"].sum()
    print(f"💰 Custo médio por inferência: ${avg_cost:.6f}")
    print(f"💰 Custo total: ${total_cost:.6f}")

print("\n🎉 Cálculo de custos concluído!")
print("💡 Agora todos os arquivos têm informações de custo incluídas.")

### **Economia da Otimização de Prompts**

#### **Calculando o Custo da Melhoria Automática de Prompts**

Além da inferência do modelo e avaliação de qualidade, nosso processo de migração aproveita o serviço de Otimização de Prompts do Amazon Bedrock pra melhorar a eficácia dos prompts entre modelos. Como outros serviços de IA, essa capacidade tem sua própria estrutura de preços que deve ser considerada no custo total da migração.

#### **Entendendo a Precificação da Otimização de Prompts**

O Bedrock cobra pela otimização de prompts baseado no volume de tokens:

- **Taxa**: $0.030 por 1.000 tokens
- **Tokens Contados**: Tanto prompts de entrada quanto prompts de saída otimizados
- **Ciclo de Cobrança**: Mensal, baseado no uso total de tokens

Esse modelo de preços significa que os custos escalam com tanto o tamanho dos seus prompts quanto o número de otimizações que você executa. Para a maioria dos cenários de migração, isso representa um pequeno custo único, mas ainda é valioso estimar para planejamento orçamentário completo.

#### **Exemplo de Cálculo**

Pra ilustrar como os custos de otimização de prompts funcionam na prática, considere este exemplo:

Um desenvolvedor de aplicação otimiza um prompt de sumarização de notícias originalmente escrito para Claude 3.5:
- Prompt original: 429 tokens
- Prompt otimizado para Claude 3.5: 511 tokens
- Este prompt otimizado é então usado como entrada para gerar variantes para:
  - Claude 3.7: 582 tokens
  - Nova Pro: 579 tokens

**Cálculo de tokens**:
- Tokens de entrada: 429 + 511 + 511 = 1.451 tokens
- Tokens de saída: 511 + 582 + 579 = 1.672 tokens
- Total de tokens: 3.123 tokens

**Cálculo de custo**:
3.123 tokens ÷ 1.000 × $0.03 = $0.09

Para nosso cenário de workshop, a otimização de prompts representa uma despesa mínima comparada aos custos contínuos de inferência, mas rastreá-la fornece uma imagem completa da economia da migração.

> **💡 Nota do Workshop**: Em sistemas de produção, a otimização de prompts pode ser executada periodicamente conforme os modelos evoluem ou os requisitos mudam. Embora o custo por otimização seja baixo, empresas com muitos prompts diferentes devem contabilizar isso em seus orçamentos operacionais.

In [ ]:
# 💰 CALCULANDO CUSTO DA OTIMIZAÇÃO DE PROMPTS
print("💰 CUSTO DA OTIMIZAÇÃO DE PROMPTS:")
print("=" * 40)

# Calculando tokens totais para otimização de prompts
input_prompt_len = 0
optimized_prompts = []

for index, evaluation in evaluation_tracking.iterrows():
    model_id = evaluation['model']
    if model_id == "source_model":
        input_prompt_len = len(evaluation['text_prompt'])
    else:
        optimized_prompts.append(evaluation['text_prompt']) 

total_prompt_len = sum(len(prompt) for prompt in optimized_prompts) + input_prompt_len * len(optimized_prompts)

# Estimativa de custo (aproximadamente 4 caracteres por token)
prompt_optimization_cost = total_prompt_len/4/1000 * 0.03

print(f"�� Tokens de entrada: {input_prompt_len}")
print(f"�� Prompts otimizados: {len(optimized_prompts)}")
print(f"📝 Total de caracteres: {total_prompt_len}")
print(f"💰 Custo estimado para otimização de prompts: ${prompt_optimization_cost:.6f}")

print("\n💡 Este é um custo único que representa a otimização dos prompts para diferentes modelos.")

### **Entendendo Custos do LLM-as-a-Judge**

#### **Quebrando a Economia da Avaliação**

Além do custo direto da inferência do modelo, é importante contabilizar a despesa da própria avaliação de qualidade. LLM-as-a-Judge é um método poderoso de avaliação, mas também tem custos associados que devem ser considerados no seu planejamento de migração.

#### **Componentes de Preços**

O Bedrock cobra pelas avaliações LLM-as-a-Judge baseado nos seguintes componentes:

1. **Inferência do Modelo**: O custo principal é pelo uso do modelo avaliador
   - Scores algorítmicos gerados automaticamente são fornecidos sem cobranças adicionais
   - Para avaliação baseada em humanos com seu próprio workstream, há uma cobrança de $0.21 por tarefa humana completada

2. **Consumo de Tokens**: Cada avaliação envolve vários componentes:
   - **Prompts do Juiz**: Cada métrica/avaliador usa seu próprio [prompt especializado](https://docs.aws.amazon.com/bedrock/latest/userguide/model-evaluation-type-judge-prompt-nova.html) (~300 tokens por métrica)
   - **Conteúdo de Entrada**: Os prompts originais e respostas do modelo sendo avaliados
   - **Resultados de Saída**: Saída JSON com scores de avaliação (~20 tokens por métrica)

#### **Fórmula de Cálculo de Custo**

Para orçamentação precisa, podemos estimar custos de avaliação usando esta fórmula para cada métrica:

Custo de Avaliação = [((Tokens nos prompts) + (Tokens nas respostas) + (Tokens do prompt do juiz))/1000 × (Preço do token de entrada do avaliador)] + [(Tokens de saída)/1000 × (Preço do token de saída do avaliador)]

Onde:
- Tokens de entrada = Número de tokens nos seus prompts + respostas + prompts do juiz (tipicamente ~300 tokens)
- Tokens de saída = Número de tokens na saída do avaliador (tipicamente ~20 tokens por métrica)

Esse entendimento detalhado dos custos de avaliação ajuda a construir uma imagem econômica completa ao comparar diferentes opções de modelo e planejar avaliação contínua de qualidade em produção.

> **💡 Nota do Workshop**: Ao projetar sua estratégia de avaliação, considere o trade-off entre avaliação abrangente (usando muitas métricas) e eficiência de custo. Para avaliações de rotina, você pode selecionar um subconjunto menor de métricas críticas para controlar custos.

In [ ]:
# 💰 CONFIGURANDO CUSTOS DO AVALIADOR
evaluator_id = "amazon.nova-pro-v1:0"

evaluator_input_price = calculator.model_input_token_prices.get(evaluator_id)
evaluator_output_price = calculator.model_output_token_prices.get(evaluator_id)

print(f"⚖️ Modelo avaliador: {evaluator_id}")
print(f"💰 Preço de entrada: ${evaluator_input_price}; preço de saída: ${evaluator_output_price}")

In [ ]:
# 💰 CALCULANDO CUSTOS DE AVALIAÇÃO DE QUALIDADE
print("💰 CUSTOS DE AVALIAÇÃO DE QUALIDADE:")
print("=" * 50)

# Verificando suas métricas de avaliação. Por padrão nosso workshop avalia 3 métricas: "Builtin.Correctness", "Builtin.Completeness", "Builtin.ProfessionalStyleAndTone"

# Encontrando todos os arquivos json correspondentes
quality_evaluation_inputs = glob.glob(os.path.join(directory, "quality_evaluation*"))

# Processando cada arquivo
total_quality_evaluation_cost = 0

for filename in quality_evaluation_inputs:
    print(f"\n📊 Processando {os.path.basename(filename)}...")
    
    evaluation_price = 0
    with open(filename, 'r') as f:
        for line in f:
            data = json.loads(line)
            
            prompt_length = len(data["prompt"])
            reference_length = len(data["referenceResponse"])
            model_response_length = len(data["modelResponses"][0]["response"])
            
            # Calculando tokens de acordo com a fórmula
            input_tokens = (prompt_length + reference_length + model_response_length) / 4 + 300  # 300 é uma estimativa aproximada, veja detalhes acima
            output_tokens = 20  # é uma estimativa aproximada
            
            # Calculando preço para esta entrada
            entry_price = (input_tokens/1000 * evaluator_input_price) + (output_tokens/1000 * evaluator_output_price)
            evaluation_price += entry_price
            
    print(f"�� Custo estimado por métrica para avaliar {os.path.basename(filename)}: ${evaluation_price:.6f}")
    total_quality_evaluation_cost += evaluation_price

## Avaliamos 3 métricas: "Builtin.Correctness", "Builtin.Completeness", "Builtin.ProfessionalStyleAndTone"
total_quality_evaluation_cost = total_quality_evaluation_cost * 3

print(f"\n💰 Custo total estimado para avaliar: ${total_quality_evaluation_cost:.6f}")
print("💡 Este custo representa a avaliação automática de qualidade usando LLM-as-a-Judge.")

### **Consolidação de Métricas de Latência e Custo de Inferência**

Nesta seção, vamos importar e processar os dados de latência coletados durante o Passo 3. Esses dados são cruciais para entender as características de performance de cada modelo, particularmente para aplicações com requisitos em tempo real ou experiências de usuário interativas.

Ao comparar essas métricas entre modelos, podemos identificar diferenças de performance que podem impactar a experiência do usuário em ambientes de produção. Isso é especialmente importante para aplicações com requisitos de latência estritos ou aquelas que servem grandes números de usuários concorrentes.

In [ ]:
# �� FUNÇÕES PRA CONSOLIDAR DADOS DE LATÊNCIA
def combine_latency_evaluation_files(directory):
    """Combina todos os arquivos CSV no diretório em um único DataFrame."""
    all_files = glob.glob(os.path.join(directory, "document_summarization_*.csv"))
    df_list = []
    for filename in all_files:
        df = pd.read_csv(filename)
        df_list.append(df)
    return pd.concat(df_list, axis=0, ignore_index=True)

def calculate_metrics(df, group_columns):
    """Calcula métricas de latência agrupadas por modelo, região e perfil de inferência."""
    metrics = df.groupby(group_columns).agg({
        'model_input_tokens': ['count', 'mean'],
        'model_output_tokens': ['mean'],
        'cost': ['mean'],
        'latency': ['mean', 'median', 
                             lambda x: x.quantile(0.9),lambda x: x.std()],
        'model_latencyMs': ['mean', 'median', 
                             lambda x: x.quantile(0.9),lambda x: x.std()]
    }).round(6)

    metrics.columns = ['sample_size', 
                      'avg_input_tokens',
                      'avg_output_tokens',
                       'avg_cost',
                       'latency_mean', 'latency_p50', 'latency_p90', 'latency_std',
                      'model_latencyMs_mean', 'model_latencyMs_p50', 'model_latencyMs_p90', 'model_latencyMs_std']
    
    metrics = metrics.reset_index()
    
    return metrics

print("✅ Funções de consolidação criadas! Vamos processar os dados.")

In [ ]:
# �� CONSOLIDANDO DADOS DE LATÊNCIA
print("📊 CONSOLIDANDO MÉTRICAS DE LATÊNCIA...")
print("=" * 50)

latency_evaluation_raw = combine_latency_evaluation_files(directory)
metrics = calculate_metrics(latency_evaluation_raw, ['model', 'region', 'inference_profile'])

print("🎯 Modelos encontrados:")
for model in metrics["model"]:
    print(f"• {model}")

display(metrics)

print("\n💡 Estas são as métricas consolidadas de latência e custo para todos os modelos.")

### **Integração de Métricas de Qualidade**

Com nossas métricas de latência e custo preparadas, agora precisamos incorporar os resultados da avaliação de qualidade dos nossos jobs LLM-as-a-Judge. Antes de analisar esses resultados, precisamos:
1. Verificar que todos os jobs de avaliação completaram
2. Localizar os arquivos de saída no nosso bucket S3
3. Extrair e analisar os scores de avaliação para cada modelo

Esses dados de qualidade completam nossa visão tridimensional da performance do modelo (latência, custo e qualidade), permitindo decisões de seleção verdadeiramente informadas.

> **⚠️ Nota do Workshop**: Jobs de avaliação podem demorar vários minutos para completar. Se seus jobs ainda estão rodando, você pode monitorar o status no console AWS ou aguardar a conclusão antes de prosseguir.

In [ ]:
# 🔧 CONFIGURANDO CLIENTES AWS
bedrock_client = boto3.client('bedrock')
s3_client = boto3.client('s3')

print("✅ Clientes AWS configurados! Vamos verificar os jobs de avaliação.")

In [ ]:
# 🔍 FUNÇÃO PRA OBTER CHAVES DE SAÍDA DO S3
import re
from collections import defaultdict

def get_s3_output_keys(evaluation_tracking):
    # Inicializando estrutura de resultado
    result = {
        "model": [],
        "key": []
    }
    
    for index, evaluation in evaluation_tracking.iterrows():
        model_id = evaluation['model']
        evaluation_job_arn = evaluation['quality_evaluation_jobArn']

        # Verificando status do job
        check_status = bedrock_client.get_evaluation_job(jobIdentifier=evaluation_job_arn)
        print(f"{model_id}: {check_status['status']}")
        
        if check_status['status'] == "Completed":
            output_path = evaluation['quality_evaluation_output']
            try:
                response = s3_client.list_objects_v2(
                    Bucket=BUCKET_NAME,
                    Prefix=PREFIX
                )

                # Encontrando o arquivo JSONL de saída para este modelo
                for obj in response.get('Contents', []):
                    key = obj['Key']
                    # Adicionando verificação de identificador do modelo
                    if key.endswith('_output.jsonl') and model_id.replace(':', '-') in key:
                        result["model"].append(model_id)
                        result["key"].append(key)
                        break
            
            except Exception as e:
                print(f"❌ Erro listando objetos para {model_id}: {str(e)}")
        else:
            print("\x1b[31mJob de avaliação de qualidade ainda está em andamento, por favor aguarde..\x1b[0m")
            sys.exit()
    
    return result

print("✅ Função de verificação criada! Vamos detectar as chaves automaticamente.")

In [ ]:
# 🔍 DETECTANDO CHAVES DE SAÍDA DO S3
print("🔍 DETECTANDO CHAVES DE SAÍDA DO S3...")
print("=" * 50)

# Uso
s3_output_keys = get_s3_output_keys(evaluation_tracking)

# Agora s3_output_keys contém um dicionário mapeando IDs de modelo para suas chaves de saída S3
print("\n�� Chaves de saída S3 detectadas automaticamente:")
for model, key in zip(s3_output_keys["model"], s3_output_keys["key"]):
    print(f"  {model}: {key}")

In [ ]:
# �� EXTRAINDO MÉTRICAS DE QUALIDADE
print("�� EXTRAINDO MÉTRICAS DE QUALIDADE...")
print("=" * 50)

### Métricas de qualidade
file_key_df = pd.DataFrame(s3_output_keys)
model_quality_list = []

for index, row in file_key_df.iterrows():  
    metrics_dict = {}

    model = row["model"]
    if row["key"] == "":
        continue
        
    response = s3_client.get_object(Bucket=BUCKET_NAME, Key=row["key"])
    content = response['Body'].read().decode('utf-8')
    
    for line in content.strip().split('\n'):
        if line:
            data = json.loads(line)
            if 'automatedEvaluationResult' in data and 'scores' in data['automatedEvaluationResult']:
                for score in data['automatedEvaluationResult']['scores']:
                    metric_name = score['metricName']
                    if 'result' in score:
                        metric_value = score['result']
                        if metric_name not in metrics_dict:
                            metrics_dict[metric_name] = []
                        metrics_dict[metric_name].append(metric_value)
    
    df = pd.DataFrame(metrics_dict)
    df['model'] = model
    model_quality_average = df.groupby("model").mean()
    model_quality_average = model_quality_average.reset_index()
    model_quality_list.append(model_quality_average)

model_quality = pd.concat(model_quality_list, axis=0, ignore_index=True)
print("�� MÉTRICAS DE QUALIDADE EXTRAÍDAS:")
print(model_quality)

print("\n💡 Estes são os scores de qualidade para cada modelo.")
print("�� Scores vão de 0 a 1, onde 1 é a melhor qualidade possível.")

> **💡 Nota do Workshop**: Esses resultados de avaliação podem não conseguir diferenciar o suficiente entre os modelos. Isso é porque temos um tamanho de amostra muito pequeno (n=10), por questão de tempo. Em implementação real, você precisará aumentar o tamanho da amostra para medir com precisão a qualidade dos modelos.

### **Análise Consolidada de Métricas**

#### **Combinando Dados de Performance, Qualidade e Custo**

Com nossas métricas de qualidade agora adicionadas aos nossos dados de performance e custo, temos uma visão completa das capacidades de cada modelo. Este framework de métricas consolidado nos permite:

1. **Comparar trade-offs**: Ver como os modelos balanceiam velocidade, qualidade e custo
2. **Identificar pontos fortes**: Determinar quais modelos se destacam em dimensões específicas
3. **Combinar com requisitos**: Alinhar capacidades com prioridades específicas da aplicação

Esta análise multidimensional é essencial para tomar decisões informadas que vão além do pensamento simplista de "melhor modelo". Diferentes aplicações têm requisitos únicos - um chatbot de atendimento ao cliente pode priorizar qualidade de resposta e tom, enquanto uma aplicação de processamento de alto volume pode favorecer velocidade e eficiência de custo.

O dataframe de métricas mescladas que criamos serve como base para nossas visualizações e relatório final, fornecendo uma imagem clara das vantagens relativas de cada opção de modelo.

In [ ]:
# �� MESCLANDO MÉTRICAS
print("�� MESCLANDO MÉTRICAS...")
print("=" * 30)

## Mesclando métricas para o dataframe de métricas
metrics = pd.merge(metrics, model_quality, on=['model'])
display(metrics)

print("\n🎉 Todas as métricas foram consolidadas em um único dataframe!")
print("💡 Agora temos latência, custo e qualidade para todos os modelos.")

### **Resumo de Análise Aprimorado por IA**

#### **Usando LLMs para Interpretar Resultados de Avaliação**

Uma das capacidades poderosas dos LLMs avançados é sua habilidade de analisar dados complexos e gerar resumos perspicazes. Nesta seção, vamos aproveitar essa capacidade perguntando ao Claude Haiku para interpretar nossos resultados de avaliação e fornecer um resumo conciso dos principais achados.

Esta abordagem demonstra um padrão importante para trabalhar com modelos foundation: usá-los não apenas como geradores de conteúdo, mas como ferramentas analíticas que podem extrair insights de dados estruturados.

O resumo resultante fornece uma interpretação legível por humanos dos nossos dados, complementando nossas visualizações e dados brutos com insights narrativos.

> **💡 Nota do Workshop**: Este padrão de "LLM como analista de dados" pode ser aplicado a muitos cenários de business intelligence além da avaliação de modelos. Considere como sua organização pode aproveitar modelos foundation para gerar insights de outros datasets complexos.

In [ ]:
# 🤖 CONFIGURANDO CLIENTE BEDROCK
client = boto3.client("bedrock-runtime")

print("✅ Cliente Bedrock configurado! Vamos gerar insights com IA.")

In [ ]:
# 🤖 GERANDO RESUMO DE ANÁLISE
print("🤖 GERANDO RESUMO DE ANÁLISE COM IA...")
print("=" * 50)

summary_prompt = """Usando o dataset fornecido abaixo, crie um resumo conciso de 3 frases que identifica qual modelo performa melhor em cada uma dessas três categorias de métricas:
1. Métricas de latência (latency_mean, latency_p50, latency_p90)
2. Métricas de custo (avg_cost)
3. Métricas de qualidade (Builtin.Correctness e completude de saída como sugerido por avg_output_tokens)
4. Não comece com "Baseado no fornecido...", apenas dê seu resumo
O resumo deve declarar claramente qual modelo é ótimo para usuários priorizando velocidade, eficiência de custo ou qualidade de saída.

Dataset: {dataset}
"""

max_tokens=1000
temperature=0
top_p=0.9

response = client.converse(
    modelId="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "text": summary_prompt.format(dataset=metrics)
                }
            ]
        }
    ],
    inferenceConfig={
        "temperature": temperature,
        "maxTokens": max_tokens,
        "topP": top_p
    }
)

analysis_summary = response['output']['message']['content'][0]['text']

print("📊 RESUMO DE ANÁLISE GERADO:")
print("-" * 40)
print(analysis_summary)

print("\n💡 Este resumo foi gerado automaticamente pela IA analisando nossos dados!")
print(" Ele nos dá uma visão rápida e objetiva dos resultados.")

### **Geração do Relatório Final**

#### **Criando Documentação de Análise Abrangente**

O culminar do nosso processo de avaliação é um relatório formatado profissionalmente que apresenta nossos achados em um formato claro e visualmente atraente. Este relatório serve múltiplos propósitos:

1. **Documentação**: Cria um registro permanente da nossa metodologia de avaliação e resultados
2. **Comunicação**: Fornece artefatos compartilháveis para discussões com stakeholders
3. **Suporte à Decisão**: Organiza informações para facilitar escolhas informadas

Nossa utilidade `generate_analysis_report` lida com o trabalho complexo de:
- Criar visualizações consistentes entre dimensões
- Formatar tabelas para legibilidade
- Gerar gráficos de distribuição de performance
- Incluir nosso resumo gerado por IA junto com métricas brutas

A saída final inclui tanto um relatório PDF quanto um resumo CSV, fornecendo opções tanto para revisões de alto nível quanto para análise detalhada.

> **💡 Nota do Workshop**: Após executar esta célula, verifique o diretório `../outputs-analysis/` para visualizar seu relatório gerado. Este relatório pode servir como template para sua própria documentação de avaliação de modelos.

In [ ]:
# 📊 GERANDO RELATÓRIO FINAL
print("📊 GERANDO RELATÓRIO FINAL...")
print("=" * 50)

report = generate_analysis_report.Analysis_Report()
report.generate_report(latency_evaluation_raw, directory, metrics, analysis_summary, total_quality_evaluation_cost, prompt_optimization_cost)

print("\n🎉 RELATÓRIO GERADO COM SUCESSO!")
print("�� Verifique o diretório ../outputs-analysis/ para ver os arquivos gerados.")
print("�� Você encontrará um relatório PDF e um resumo CSV.")

### **Resumo e Principais Takeaways**

#### **Parabéns por Completar o Workshop de Avaliação de Modelos!**

Você navegou com sucesso por todo o processo de avaliação e migração de modelos, ganhando experiência prática com uma metodologia que pode ser aplicada aos seus próprios projetos GenAI. Neste notebook final, você:

1. ✅ **Consolidou métricas multidimensionais** entre latência, qualidade e custo
2. ✅ **Calculou implicações econômicas** de diferentes escolhas de modelo em escala  
3. ✅ **Analisou avaliações de qualidade** das avaliações LLM-as-a-Judge
4. ✅ **Gerou insights aprimorados por IA** para interpretar dados complexos de avaliação
5. ✅ **Criou documentação profissional** para apoiar tomada de decisão

### **Principais Takeaways**

1. **Seleção de Modelo é Multidimensional**: Raramente há um único modelo "melhor" - diferentes modelos se destacam em áreas diferentes, e a escolha ótima depende dos seus requisitos específicos.
2. **Migração Baseada em Dados**: Migrações bem-sucedidas de modelos requerem medições objetivas em vez de apenas suposições ou especificações.
3. **Análise de Trade-offs**: Entender a relação entre velocidade, qualidade e custo permite decisões informadas que balanceiam prioridades concorrentes.
4. **Documentação Importa**: Documentação completa da sua metodologia de avaliação e resultados ajuda a construir consenso e apoiar decisões futuras de migração.
5. **Avaliação Contínua**: Conforme novos modelos são lançados e existentes são atualizados, o processo de avaliação deve ser repetido periodicamente para garantir que você está usando componentes ótimos.

### **Próximos Passos**

Considere aplicar este framework de avaliação aos seus próprios casos de uso:
- **Personalize critérios de sucesso**: Defina thresholds específicos de aceitação baseados nos requisitos da sua aplicação
- **Amplie a comparação**: Avalie mais modelos entre diferentes provedores e arquiteturas
- **Expanda as métricas**: Adicione critérios de avaliação específicos do domínio relevantes para suas aplicações
- **Continue otimização de prompts**: Lembre-se que engenharia de prompts é um esforço contínuo - considere construir um pipeline sistemático para testar variações de prompts com métricas automatizadas ou avaliação human-in-the-loop
- **Explore mais opções de Inferência**: Aprenda e explore mais opções de otimização de inferência no Bedrock que podem melhorar a latência para seus casos de uso, ex: [Inferência Otimizada](https://docs.aws.amazon.com/bedrock/latest/userguide/latency-optimized-inference.html), [Cache de Prompts](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-caching.html), [Perfil de Inferência](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles.html) e outros.
- **Construa componente de avaliação RAG**: Se seu caso de uso requer uma base de conhecimento construída com RAG, considere adicionar um componente de avaliação de qualidade para RAG. Opções de avaliação RAG incluem [Bedrock RAG evaluator](https://docs.aws.amazon.com/bedrock/latest/userguide/evaluation-kb.html), soluções open source como [RAGAS](https://docs.ragas.io/en/stable/getstarted/rag_eval/). 
- **Automatize o workflow**: Integre estes passos de avaliação no seu pipeline CI/CD para avaliação contínua de modelos
- **Estabeleça loops de feedback**: Crie mecanismos para capturar dados de performance de produção para informar futuras otimizações de prompts e modelos

Obrigado por participar deste workshop! Esperamos que as habilidades e metodologia que você aprendeu ajudem você a tomar decisões confiantes e baseadas em dados sobre seleção e migração de modelos na sua jornada GenAI.

---

**💡 Dica Final do Pedro**: Lembre-se, migração de modelo não é uma corrida - é uma maratona! Tome seu tempo, teste bem, e sempre mantenha a qualidade em mente. É melhor ter um modelo um pouco mais lento que funciona perfeitamente do que um super rápido que quebra toda hora!

**�� Próximo passo**: Aplicar esta metodologia aos seus próprios projetos!